In [1]:
import pickle
import numpy as np
import pandas as pd
from pprint import pprint
from time import time
import logging
import sys
import tensorflow as tf
from os import path
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow import feature_column
import tensorflow_hub as hub

state_gov_nm = 'CA'
year_to_process = '2020'
data_file_location = 'C:/Users/William Mitchell/Documents/QuigleyData'

#state_gov_nm = sys.argv[1]
#year_to_process = sys.argv[2]
#data_file_location = sys.argv[3]



In [2]:
#Create features for text
#module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"

#Model load through local path:
#module_path = data_file_location + '/universal-sentence-encoder_4'
#%time embed = hub.load(module_path)

In [3]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [4]:
leg_data_raw = pd.DataFrame()
legcorpus = []
win_state = []
_legtext = path.join(data_file_location,state_gov_nm + 'embedded_legdoc.df')
_bill_clusters = path.join(data_file_location,state_gov_nm + 'bill_clusters.df')
_sponsors = path.join(data_file_location, state_gov_nm + 'Sponsors.csv') 
_cmltv_event_df = path.join(data_file_location,state_gov_nm + 'cmltv_events.df')

In [5]:
#Temp, delete this after US testing
for i in range(2009, int(year_to_process)+1):

    try:
        _WinList = path.join(data_file_location, state_gov_nm + str(i) + 'WinList.list')
        
        _LegOverallData_CSV = path.join(data_file_location, state_gov_nm + str(i) + 'LegOverallData.csv')
        
        leg_data_raw = leg_data_raw.append(pd.read_csv(_LegOverallData_CSV))
        
        with open(_WinList,'rb') as f:
            win_state.extend(pickle.load(f))

    except:
        continue

In [6]:
for i in range(2009, int(year_to_process)+1):

    try:
        _WinList = path.join(data_file_location, state_gov_nm + str(i) + 'WinList.list')
        
        _LegOverallData_CSV = path.join(data_file_location, state_gov_nm + str(i) + 'LegOverallData.csv')
        
        leg_data_raw = leg_data_raw.append(pd.read_csv(_LegOverallData_CSV))
        
        with open(_WinList,'rb') as f:
            win_state.extend(pickle.load(f))

    except:
        continue

with open(_legtext,'rb') as f:
    legtextembed = pickle.load(f)

with open(_cmltv_event_df,'rb') as f:
    cmltv_event = pickle.load(f)

with open(_bill_clusters,'rb') as f:
    billcat = pickle.load(f)
    
#sponsor_df = pd.read_csv(_sponsors)
#sponsor_df.set_index('bill_id', in_place = True)


In [7]:
leg_data = leg_data_raw

In [8]:
combined_docs = {}
total = 0
passed = 0
leg_target = {}
bill_order = []
bill_target = []
#for each document, check if it is approved, record that, then combine the list into a string
#for doc in legcorpus:
#    
#    pass_flag = 0
#    if doc[0] not in combined_docs:
##        total += 1
##        try:
##           combined_docs[doc[0]] = " ".join(doc[2])
##        except:
##            pass

leg_targ = pd.DataFrame({"every stage passed":win_state})

for i, bill_id in enumerate(leg_data['bill_id']):
    if str(bill_id) not in bill_order:
        bill_order.append(str(bill_id))
        bill_target.append(leg_targ["every stage passed"][i])
        leg_target[str(bill_id)] = leg_targ["every stage passed"][i]
        

In [9]:
#target list contains NaN entries and also is floating point.  Conversion to integer.#####
leg_data['unclean_label'] = leg_targ

leg_data['clean_label'] = leg_data['unclean_label'].fillna(0.0)

int_label = []

for i in leg_data['clean_label']:
    int_label.append(int(i))

leg_data['target'] = int_label

##########################################################################################
#Clean up dataframes
legtextembed = legtextembed.drop_duplicates(subset=['bill_id'])
legtextembed = legtextembed.set_index('bill_id')
billcat['bill_id'] = billcat['bill_id'].astype('int64')
#leg_data = leg_data.drop_duplicates(subset=['bill_id']) #should probably be bill_id
#leg_data = leg_data.set_index('bill_id')
leg_data = leg_data.drop(columns=['event', 'Unnamed: 0', 'primary_id','title'])

In [10]:

leg_data['chamber'] = leg_data['chamber'].fillna('None')
leg_data['primary_name'] = leg_data['primary_name'].fillna('None')
leg_data['primary_party'] = leg_data['primary_party'].fillna('N')

#adjust the number of sponsors, based on the max value found in the field (for future iterations, just divide by the number in that particular chamber)
max_sponsors = leg_data['number_sponsors'].max()
leg_data['num_sponsors_adj'] = (leg_data['number_sponsors']*1.00)/max_sponsors

#filtering out all types of legislation except bills (no resolutions, constitutional ammendments, etc.)
leg_data = leg_data[leg_data['bill_type']=='B']

print(len(leg_data), len(legtextembed))

#leg_data = leg_data.join(legtextembed)

leg_data = pd.merge(leg_data, billcat, how='inner', on=['bill_id'])
leg_data = pd.merge(leg_data, cmltv_event, how='inner', on=['bill_id','time_t'])
#leg_data = leg_data.join(sponsor_df, how='inner')
#leg_data['category'] = leg_data['category'].fillna(int(51))


863872 30967


In [11]:
#bill_texts = []
#create the list of bill texts
#for bill_id in leg_data.index:
#    if bill_id in combined_docs:
#        bill_texts.append(combined_docs[bill_id])
#    else:
#        bill_texts.append("")
#
#leg_data['texts'] = bill_texts


In [12]:
#next, embed the text using Google's DAN
#text_embedding = []
#i=0
#for t in bill_texts:
#embed_to_append = embed(bill_texts)
#convert tensor to an array, may not need later, needed to keep dataset conversion from breaking
#text_embedding.extend(embed_to_append)
#leg_data['embedded_text'] = text_embedding

In [13]:
train_set = leg_data[leg_data['year']<int(year_to_process)]
test_set = leg_data[leg_data['year']==int(year_to_process)]
#prediction_set = leg_data[leg_data['year']==int(year_to_process)]

test = test_set
train, val = train_test_split(train_set, test_size=.25)

#train, test = train_test_split(leg_data, test_size=0.2)
#train, val = train_test_split(leg_data, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

554790 train examples
184930 validation examples
124152 test examples


In [14]:
#remove all but the latest entries for the test data set
#this will grab the highest time_t for a given bill_id and then apply the index of that to the original data frame
#This will take our test data and only look at the latest status of it to determine pass or failure
#only do this for the prediction set!!!!! not the testing set as those ran the full course.  For that we need a random sampling as a test.

idx = test.groupby(['bill_id'])['time_t'].transform(max) == test['time_t']
test = test[idx]
print(len(test))


9696


In [15]:
#print(leg_data['embedded_text'].iloc[0:5])
#for
#unested = [x for x in leg_data['embedded_text']]
#print(unested[0:1])

In [16]:
#batch_size = 5 # A small batch sized is used for demonstration purposes
#train_ds = df_to_dataset(train, batch_size=batch_size)
#val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
#test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [17]:
#for feature_batch, label_batch in train_ds.take(1):
#  print('Every feature:', list(feature_batch.keys()))
#  for l in list(feature_batch.keys()):  
#    print(feature_batch[l])
#  print('A batch of targets:', label_batch )

In [18]:
# We will use this batch to demonstrate several types of feature columns
#example_batch = next(iter(train_ds))[0]

In [19]:
#def demo(feature_column):
#  feature_layer = layers.DenseFeatures(feature_column)
#  print(feature_layer(example_batch).numpy())

In [20]:
#primary_sponsor = feature_column.categorical_column_with_vocabulary_list('primary_name', leg_data['primary_name'].unique())
#primary_embedding = feature_column.embedding_column(primary_sponsor, dimension=8)
#demo(primary_embedding)

In [21]:
feature_columns = []

numericfeaturelist = ['sponsor_vec'
                      , 'num_sponsors_adj'
                      #, 'intro_to_end_of_session'
                      #, 'last_event_to_end_of_session'
                      ]

#for dv in range(1,21):
#  numericfeaturelist.append('dv' + str(dv))

#numeric cols
for header in numericfeaturelist:
  feature_columns.append(feature_column.numeric_column(header))

# bucketized cols
#age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
#feature_columns.append(age_buckets)

# indicator cols
primary_party = feature_column.categorical_column_with_vocabulary_list('primary_party', leg_data['primary_party'].unique())
primary_party_onehot = feature_column.indicator_column(primary_party)
feature_columns.append(primary_party_onehot)

time_t = feature_column.categorical_column_with_vocabulary_list('time_t', leg_data['time_t'].unique())
time_t_onehot = feature_column.indicator_column(time_t)
feature_columns.append(time_t_onehot)

primary_sponsor = feature_column.categorical_column_with_vocabulary_list('primary_name', leg_data['primary_name'].unique())
primary_sponsor_onehot = feature_column.indicator_column(primary_sponsor)
feature_columns.append(primary_sponsor_onehot)

bill_category = feature_column.categorical_column_with_vocabulary_list('category', leg_data['category'].unique())
bill_category_onehot = feature_column.indicator_column(bill_category)
feature_columns.append(bill_category_onehot)

# embedding cols
event_list = feature_column.categorical_column_with_vocabulary_list('cmltv_event', leg_data['cmltv_event'].unique())
event_list_onehot = feature_column.indicator_column(event_list)
event_list_embed = feature_column.embedding_column(event_list, dimension=16)
#feature_columns.append(event_list_embed)
feature_columns.append(event_list_onehot)
#primary_embedding = feature_column.embedding_column(primary_sponsor, dimension=8)




In [22]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [23]:
batch_size = 512
train_ds = df_to_dataset(train, shuffle = True, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=True, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [ ]:
#activation = ['relu','sigmoid','softmax']
#dropout = [.1,.2]
#optimizer = ['adam','sgd','adamax']

#a= 'relu'
#d = .1
#o = 'adam'

#print(a,d,o)
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(35, activation='relu'),
  #layers.Dense(128, activation='relu'),
  #layers.Dropout(.1),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          #validation_data = test_ds,
          validation_data=val_ds,
          epochs=50)

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

In [ ]:
predictions = model.predict(test_ds)

In [ ]:
import math
pred_list = []
for p in predictions:
    s_pred = 1/(1+math.exp(-p))
    pred_list.append(s_pred)

In [ ]:
i=0
for s in pred_list:
    if s > .5:
        i += 1
print(i/len(pred_list))

In [ ]:
pred_bill_ids = test_ds.take(1)

In [ ]:
test_set['pred'] = pred_list

In [ ]:
test_set.to_csv('test.csv')